In [ ]:
import pandas as pd
label = pd.read_csv("/home/iatell/projects/meta-learning/data/seq_line_labels.csv")
label["seq_len"] = label["endIndex"] - label["startIndex"]
label

In [2]:

import pandas as pd
df = pd.read_csv("/home/iatell/projects/meta-learning/data/Bitcoin_BTCUSDT_kaggle_1D_candles_prop.csv")
df

,timestamp,open,high,low,close,volume,upper_shadow,body,lower_shadow,Candle_Color,upper_body_ratio,lower_body_ratio,upper_lower_body_ratio
0,2018-01-01,13707.91,13818.55,12750.00,13380.00,8607.15640,0.076003,-0.225254,0.432772,1,0.337410,1.921259,0.175619
1,2018-01-02,13382.16,15473.49,12890.02,14675.11,20078.16540,0.540071,0.874627,0.332912,2,0.617487,0.380633,1.622262
2,2018-01-03,14690.00,15307.56,14150.00,14919.51,15905.48210,0.263644,0.155931,0.366880,2,1.690776,2.352839,0.718611
3,2018-01-04,14919.51,15280.00,13918.04,15059.54,25224.41500,0.150006,0.095280,0.681423,2,1.574377,5.000000,0.220136
4,2018-01-05,15059.56,17176.24,14600.00,16960.39,23251.35200,0.144690,1.274181,0.308056,2,0.113556,0.241768,0.469688
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1599,2022-05-19,28715.33,30545.18,28691.38,30319.23,67877.36415,0.109006,0.773779,0.011554,2,0.140875,0.014932,5.000000
1600,2022-05-20,30319.22,30777.33,28730.00,29201.01,60517.25325,0.221063,-0.539597,0.227288,1,0.409682,0.421218,0.972612
1601,2022-05-21,29201.01,29656.18,28947.28,29445.06,20987.13124,0.103235,0.119338,0.124071,2,0.865069,1.039664,0.832066
1602,2022-05-22,29445.07,30487.99,29255.11,30293.94,36158.98748,0.095648,0.418411,0.093632,2,0.228598,0.223780,1.021531


# model


## two head lstm

In [6]:
import torch
import torch.nn as nn
import pytorch_lightning as pl
from torch.nn.utils.rnn import pack_padded_sequence

class LSTMMultiRegressor(pl.LightningModule):
    def __init__(self, input_dim, hidden_dim, num_layers, max_len_y, lr=0.001, threshold=0.5):
        super().__init__()
        self.save_hyperparameters()

        self.lstm = nn.LSTM(
            input_size=input_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True,
        )
        # Main regression output: predict all linePrices up to max_len_y
        self.fc_reg = nn.Linear(hidden_dim, max_len_y)
        # Length prediction branch: logits per possible line (max_len_y)
        self.fc_len = nn.Linear(hidden_dim, max_len_y)
        self.lr = lr
        self.threshold = threshold

        self.loss_fn_reg = nn.MSELoss(reduction="none")  # we'll mask padded values
        self.loss_fn_len = nn.BCEWithLogitsLoss()        # treat as multi-label classification

    def forward(self, x, lengths):
        x = x["main"] 
        packed = pack_padded_sequence(x, lengths.cpu(), batch_first=True, enforce_sorted=False)
        _, (hn, _) = self.lstm(packed)
        last_h = hn[-1]

        y_pred = self.fc_reg(last_h)      # regression outputs
        len_logits = self.fc_len(last_h)  # logits per possible line
        return y_pred, len_logits

    def training_step(self, batch, batch_idx):
        X, y, lengths = batch
        y_pred, len_logits = self(X, lengths)

        # --- Regression loss with masking ---
        mask = (y != 0).float()  # assume padding = 0
        loss_reg = (self.loss_fn_reg(y_pred, y) * mask).sum() / mask.sum()

        # --- Length loss ---
        target_lengths = torch.zeros_like(len_logits, dtype=torch.float32)
        for i, l in enumerate(lengths):
            target_lengths[i, :l] = 1.0   # first l positions are 1, rest are 0

        loss_len = self.loss_fn_len(len_logits, target_lengths)

        loss = loss_reg + 0.1 * loss_len
        self.log("train_loss", loss, prog_bar=True)
        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.lr)

    def predict_length(self, len_logits):
        """
        Convert logits to predicted number of lines using threshold.
        """
        probs = torch.sigmoid(len_logits)
        pred_len = (probs > self.threshold).sum(dim=1)
        return pred_len


## FNNCNN

In [ ]:
import torch
import torch.nn as nn
import pytorch_lightning as pl
import torch.nn.functional as F
from torch.nn.utils.rnn import pack_padded_sequence

def mdn_split_params(raw_params, n_components):
    """
    raw_params: (B, 3K) tensor from mdn_head
    returns:
        pi    (B, K) mixture weights
        mu    (B, K) means
        sigma (B, K) std devs
    """
    B, threeK = raw_params.shape
    assert threeK == 3 * n_components

    raw = raw_params.view(B, n_components, 3)

    pi = raw[..., 0]                 # (B,K)
    mu = raw[..., 1]                 # (B,K)
    sigma = raw[..., 2]              # (B,K)

    pi = F.softmax(pi, dim=-1)       # weights sum to 1
    sigma = F.softplus(sigma) + 1e-4 # strictly positive
    return pi, mu, sigma


def mdn_nll_multitarget(y_line, pi, mu, sigma):
    """
    Negative log-likelihood for MDN with multiple valid targets per sample.
    Args:
        y_line : (B, L) padded targets (0 where invalid)
        pi, mu, sigma : (B, K) MDN params
    Returns:
        scalar loss
    """
    B, K = mu.shape
    losses = []

    for b in range(B):
        valid_y = y_line[b][y_line[b] > 0]  # (M,)
        if len(valid_y) == 0:
            continue

        # expand to (M, K)
        y_exp = valid_y.unsqueeze(-1).expand(-1, K)

        log_prob = -0.5 * ((y_exp - mu[b]) / (sigma[b] + 1e-8))**2 \
                   - torch.log(sigma[b] + 1e-8) \
                   - 0.5 * torch.log(torch.tensor(2.0 * torch.pi, device=y_line.device))

        log_mix = torch.log(pi[b] + 1e-8) + log_prob
        log_sum = torch.logsumexp(log_mix, dim=-1)  # (M,)

        losses.append(-log_sum.mean())

    if len(losses) == 0:
        return torch.tensor(0.0, device=y_line.device, requires_grad=True)

    return torch.stack(losses).mean()


class CNNLSTM_MDN(pl.LightningModule):
    def __init__(self, input_dim, hidden_dim=128, num_layers=1, hidden_features=64, out_features=32,
                 lr=1e-3, n_components=5, cnn_channels=64, dropout=0.1):
        super().__init__()
        self.save_hyperparameters()

        # Time-distributed feature extractor
        self.fc1 = nn.Linear(input_dim, hidden_features)
        self.ln1 = nn.LayerNorm(hidden_features) # ADDED: LayerNorm for time-step features
        self.fc2 = nn.Linear(hidden_features, out_features)
        self.ln2 = nn.LayerNorm(out_features) # ADDED: LayerNorm

        # CNN feature extractors
        self.conv1 = nn.Conv1d(out_features, cnn_channels, kernel_size=1)
        self.bn1 = nn.BatchNorm1d(cnn_channels) # ADDED: BatchNorm for convolutional features
        self.conv3 = nn.Conv1d(out_features, cnn_channels, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm1d(cnn_channels) # ADDED: BatchNorm

        # Learnable mixer for CNN outputs
        self.mixer = nn.Conv2d(in_channels=2, out_channels=1, kernel_size=1, bias=True)

        # LSTM for temporal dependency
        fused_dim = cnn_channels # Input to LSTM is the mixed CNN output
        self.lstm = nn.LSTM(fused_dim, hidden_dim, num_layers=num_layers,
                            batch_first=True, dropout=dropout if num_layers > 1 else 0)

        # MDN Head
        self.mdn_head = nn.Linear(hidden_dim, 3 * n_components)
        self.n_components = n_components
        self.lr = lr

        # Apply weight initialization
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            nn.init.kaiming_normal_(module.weight, mode="fan_in", nonlinearity="relu")
            if module.bias is not None:
                nn.init.constant_(module.bias, 0)
        elif isinstance(module, (nn.Conv1d, nn.Conv2d)):
            nn.init.kaiming_normal_(module.weight, mode="fan_out", nonlinearity="relu")
            if module.bias is not None:
                nn.init.constant_(module.bias, 0)

    def forward(self, X, lengths=None):
        # Input shape X["main"]: (B, T, F_in)
        x = X["main"] # REMOVED redundant transposes

        # 1. Time-distributed feature extraction
        x = self.fc1(x)
        x = F.relu(self.ln1(x)) # CHANGED: Apply LayerNorm before ReLU
        x = self.fc2(x)
        x = F.relu(self.ln2(x)) # CHANGED: Apply LayerNorm before ReLU

        # 2. CNN feature extraction
        x = x.transpose(1, 2)   # Shape: (B, C_in, L=T)
        x1 = F.relu(self.bn1(self.conv1(x))) # CHANGED: Apply BatchNorm before ReLU
        x3 = F.relu(self.bn3(self.conv3(x))) # CHANGED: Apply BatchNorm before ReLU

        # 3. Mix CNN outputs
        paired = torch.stack([x1, x3], dim=1) # Shape: (B, 2, C_out, L)
        mixed = self.mixer(paired)            # Shape: (B, 1, C_out, L)
        
        # Prepare for LSTM
        xf = mixed.squeeze(1).transpose(1, 2) # Shape: (B, L, C_out)

        # 4. LSTM for sequence summary
        if lengths is not None:
            packed_input = pack_padded_sequence(
                xf, lengths.cpu(), batch_first=True, enforce_sorted=False
            )
            _, (h_last, _) = self.lstm(packed_input)
        else:
            _, (h_last, _) = self.lstm(xf)
        
        last_h = h_last[-1] # Shape: (B, H)
        
        # 5. MDN head for distribution parameters
        raw = self.mdn_head(last_h)
        pi, mu, sigma = mdn_split_params(raw, self.n_components)
        return {"pi": pi, "mu": mu, "sigma": sigma}

    def training_step(self, batch, batch_idx):
        X, y_line, lengths = batch
        mdn = self(X, lengths)
        loss = mdn_nll_multitarget(y_line, mdn["pi"], mdn["mu"], mdn["sigma"])
        self.log("train/loss", loss)
        return loss

    def validation_step(self, batch, batch_idx):
        X, y_line, lengths = batch
        mdn = self(X, lengths)
        loss = mdn_nll_multitarget(y_line, mdn["pi"], mdn["mu"], mdn["sigma"])
    # Log everything to progress bar
        self.log("val/loss", loss, prog_bar=True)
        self.log("val/pi_mean", mdn["pi"].mean(), prog_bar=True)
        self.log("val/pi_std", mdn["pi"].std(), prog_bar=True)
        self.log("val/mu_mean", mdn["mu"].mean(), prog_bar=True)
        self.log("val/mu_std", mdn["mu"].std(), prog_bar=True)
        self.log("val/sigma_mean", mdn["sigma"].mean(), prog_bar=True)
        self.log("val/sigma_std", mdn["sigma"].std(), prog_bar=True)
        
    def configure_optimizers(self): 
        return torch.optim.Adam(self.parameters(), lr=self.lr)
    # def configure_optimizers(self):
    #     optimizer = torch.optim.Adam(self.parameters(), lr=self.lr)
    #     scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    #         optimizer,
    #         mode="min",
    #         factor=0.2,   # Reduce LR by 80%
    #         patience=5,   # After 5 epochs of no val_loss improvement
    #         verbose=True
    #     )
    #     return {
    #         "optimizer": optimizer,
    #         "lr_scheduler": {
    #             "scheduler": scheduler,
    #             "monitor": "val/loss",  # Important!
    #         },
        # }


## CNNLSTM weightening

In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.distributions import Normal
import pytorch_lightning as pl
# Your mdn_split_params function remains the same
def mdn_split_params(raw_params, n_components):
    B, threeK = raw_params.shape
    assert threeK == 3 * n_components
    raw = raw_params.view(B, n_components, 3)
    pi = F.softmax(raw[..., 0], dim=-1)
    mu = raw[..., 1]
    sigma = F.softplus(raw[..., 2]) + 1e-4
    return pi, mu, sigma

def weighted_mdn_nll(y_true, mdn_params, weights):
    total_loss = 0.0
    num_lines = y_true.shape[1]
    B = y_true.shape[0]

    # Keep track if any valid lines are found
    valid_line_found = False

    for i in range(num_lines):
        y_target = y_true[:, i:i+1]  # (B,1)
        pi, mu, sigma = mdn_params['pi'][i], mdn_params['mu'][i], mdn_params['sigma'][i]

        mask = (y_target != 0).squeeze()
        if mask.sum() == 0:
            continue

        valid_line_found = True
        y_target_masked = y_target[mask]
        pi_masked, mu_masked, sigma_masked = pi[mask], mu[mask], sigma[mask]

        dist = Normal(loc=mu_masked, scale=sigma_masked)
        log_prob = dist.log_prob(y_target_masked.expand_as(mu_masked))
        log_mix_prob = torch.log(pi_masked + 1e-8) + log_prob
        log_likelihood = torch.logsumexp(log_mix_prob, dim=1)
        line_loss = -log_likelihood.mean()
        total_loss += weights[i] * line_loss

    if not valid_line_found:
        # Avoid returning a Python float; create a tensor with requires_grad
        total_loss = torch.tensor(0.0, device=y_true.device, requires_grad=True)

    return total_loss


class CNNLSTM_MDN_MultiHead(pl.LightningModule):
    def __init__(self, input_dim, hidden_dim=128, num_layers=1, hidden_features=64, out_features=32,
                 lr=1e-3, n_components=5, cnn_channels=64, dropout=0.1, num_lines=9):
        super().__init__()
        self.save_hyperparameters()

        # --- Your CNN and LSTM base remains the same ---
        self.fc1 = nn.Linear(input_dim, hidden_features)
        self.ln1 = nn.LayerNorm(hidden_features)
        self.fc2 = nn.Linear(hidden_features, out_features)
        self.ln2 = nn.LayerNorm(out_features)
        self.conv1 = nn.Conv1d(out_features, cnn_channels, kernel_size=1)
        self.bn1 = nn.BatchNorm1d(cnn_channels)
        self.conv3 = nn.Conv1d(out_features, cnn_channels, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm1d(cnn_channels)
        self.mixer = nn.Conv2d(in_channels=2, out_channels=1, kernel_size=1, bias=True)
        fused_dim = cnn_channels
        self.lstm = nn.LSTM(fused_dim, hidden_dim, num_layers=num_layers,
                              batch_first=True, dropout=dropout if num_layers > 1 else 0)

        # === MODIFICATION: Create a list of MDN heads ===
        self.num_lines = num_lines
        self.mdn_heads = nn.ModuleList(
            [nn.Linear(hidden_dim, 3 * n_components) for _ in range(num_lines)]
        )

        self.n_components = n_components
        self.lr = lr

        # === Define importance weights here ===
        # Using exponential decay: w_i = 0.9^(i-1)
        weights = torch.tensor([0.9**i for i in range(self.num_lines)])
        self.register_buffer('loss_weights', weights)

        self.apply(self._init_weights)

    def _init_weights(self, module): # Your init function is fine
        if isinstance(module, nn.Linear):
            nn.init.kaiming_normal_(module.weight, mode="fan_in", nonlinearity="relu")
            if module.bias is not None: nn.init.constant_(module.bias, 0)
        elif isinstance(module, (nn.Conv1d, nn.Conv2d)):
            nn.init.kaiming_normal_(module.weight, mode="fan_out", nonlinearity="relu")
            if module.bias is not None: nn.init.constant_(module.bias, 0)

    def forward(self, X, lengths=None):
        # --- Your forward pass for the base model is the same ---
        x = X["main"]
        x = F.relu(self.ln1(self.fc1(x)))
        x = F.relu(self.ln2(self.fc2(x)))
        x = x.transpose(1, 2)
        x1 = F.relu(self.bn1(self.conv1(x)))
        x3 = F.relu(self.bn3(self.conv3(x)))
        paired = torch.stack([x1, x3], dim=1)
        mixed = self.mixer(paired)
        xf = mixed.squeeze(1).transpose(1, 2)
        
        # We'll assume lengths is None for simplicity here, but your implementation is fine
        _, (h_last, _) = self.lstm(xf)
        last_h = h_last[-1]

        # === MODIFICATION: Get parameters from all heads ===
        all_params = {'pi': [], 'mu': [], 'sigma': []}
        for i in range(self.num_lines):
            raw_params = self.mdn_heads[i](last_h)
            pi, mu, sigma = mdn_split_params(raw_params, self.n_components)
            all_params['pi'].append(pi)
            all_params['mu'].append(mu)
            all_params['sigma'].append(sigma)

        return all_params
    

    # This would be inside your CNNLSTM_MDN_MultiHead class

    def training_step(self, batch, batch_idx):
        # Assuming your batch now provides a y tensor of shape (B, 9)
        # where y has your target line values, padded with -1.
        X, y, lengths = batch

        # Get the dictionary of parameter lists from the forward pass
        mdn_params = self(X, lengths)

        # Calculate loss using the new weighted function
        loss = weighted_mdn_nll(y, mdn_params, self.loss_weights)

        self.log("train/loss", loss)
        return loss

    def configure_optimizers(self):
        return torch.optim.AdamW(self.parameters(), lr=self.lr)

    # NOTE: You'll also need a validation_step that mirrors the training_step logic
    def validation_step(self, batch, batch_idx):
        X, y, lengths = batch
        mdn_params = self(X, lengths)
        loss = weighted_mdn_nll(y, mdn_params, self.loss_weights)
        self.log("val/loss", loss, prog_bar=True)
        return loss

## CNNLSTM weightening with pi order

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.distributions import Normal
import pytorch_lightning as pl


def mdn_split_params(raw_params, n_components):
    """
    Splits raw MDN output into mixture weights (pi), means (mu), and stds (sigma)
    """
    B, threeK = raw_params.shape
    assert threeK == 3 * n_components
    raw = raw_params.view(B, n_components, 3)
    pi = F.softmax(raw[..., 0], dim=-1)           # mixture probabilities
    mu = raw[..., 1]                              # means
    sigma = F.softplus(raw[..., 2]) + 1e-4       # stds
    return pi, mu, sigma


def weighted_mdn_nll(y_true, mdn_params, weights):
    """
    y_true: (B, num_lines)
    mdn_params: dict with 'pi', 'mu', 'sigma' each of shape (B, n_components)
    weights: (num_lines,) tensor
    """
    B, num_lines = y_true.shape
    pi, mu, sigma = mdn_params['pi'], mdn_params['mu'], mdn_params['sigma']  # (B, n_components)

    # Sort components by pi descending
    _, idx = torch.sort(pi, descending=True, dim=1)  # (B, n_components)

    total_loss = 0.0
    valid_line_found = False

    for i in range(num_lines):
        y_target = y_true[:, i]  # (B,)

        # Skip masked/padded targets
        mask = (y_target != 0)
        if mask.sum() == 0:
            continue
        valid_line_found = True

        # Select top pi component for this line
        top_mu = mu.gather(1, idx[:, i].unsqueeze(1)).squeeze(1)      # (B,)
        top_sigma = sigma.gather(1, idx[:, i].unsqueeze(1)).squeeze(1) # (B,)
        y_target_masked = y_target[mask]
        top_mu_masked = top_mu[mask]
        top_sigma_masked = top_sigma[mask]

        dist = Normal(top_mu_masked, top_sigma_masked)
        line_loss = -dist.log_prob(y_target_masked).mean()
        total_loss += weights[i] * line_loss

    if not valid_line_found:
        total_loss = torch.tensor(0.0, device=y_true.device, requires_grad=True)

    return total_loss


class CNNLSTM_MDN(pl.LightningModule):
    def __init__(self, input_dim, hidden_dim=128, n_components=9, num_lines=9, lr=1e-3, dropout=0.1):
        super().__init__()
        self.save_hyperparameters()
        self.num_lines = num_lines
        self.n_components = n_components
        self.lr = lr

        # Base network
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.ln1 = nn.LayerNorm(hidden_dim)
        self.lstm = nn.LSTM(hidden_dim, hidden_dim, batch_first=True, dropout=dropout)

        # Single MDN head predicting n_components Gaussians
        self.mdn_head = nn.Linear(hidden_dim, 3 * n_components)

        # Importance weights for lines
        weights = torch.tensor([0.9**i for i in range(num_lines)], dtype=torch.float)
        self.register_buffer("loss_weights", weights)

        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            nn.init.kaiming_normal_(module.weight, mode="fan_in", nonlinearity="relu")
            if module.bias is not None: nn.init.constant_(module.bias, 0)

    def forward(self, X, lengths=None):
        """
        X: (B, T, input_dim)
        """
        x = X["main"]
        x = F.relu(self.ln1(self.fc1(x)))
        
        if lengths is not None:
            x = nn.utils.rnn.pack_padded_sequence(x, lengths.cpu(), batch_first=True, enforce_sorted=False)
            _, (h_last, _) = self.lstm(x)
        else:
            _, (h_last, _) = self.lstm(x)

        last_h = h_last[-1]  # (B, hidden_dim)
        raw_params = self.mdn_head(last_h)  # (B, 3*n_components)
        pi, mu, sigma = mdn_split_params(raw_params, self.n_components)
        return {"pi": pi, "mu": mu, "sigma": sigma}

    def training_step(self, batch, batch_idx):
        X, y, lengths = batch
        mdn_params = self(X)
        loss = weighted_mdn_nll(y, mdn_params, self.loss_weights)
        self.log("train/loss", loss)
        return loss

    def validation_step(self, batch, batch_idx):
        X, y, lengths = batch
        mdn_params = self(X)
        loss = weighted_mdn_nll(y, mdn_params, self.loss_weights)
        self.log("val/loss", loss, prog_bar=True)
        return loss

    def configure_optimizers(self):
        return torch.optim.AdamW(self.parameters(), lr=self.lr)


## CNNLSTM weightening with sigma confidance

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.distributions import Normal
import pytorch_lightning as pl
# Your mdn_split_params function remains the same
def mdn_split_params(raw_params, n_components):
    B, threeK = raw_params.shape
    assert threeK == 3 * n_components
    raw = raw_params.view(B, n_components, 3)
    pi = F.softmax(raw[..., 0], dim=-1)
    mu = raw[..., 1]
    sigma = F.softplus(raw[..., 2]) + 1e-4
    return pi, mu, sigma

def weighted_mdn_nll_with_sigma_penalty(y_true, mdn_params, weights, lambda_sigma=0.01):
    """
    Calculates weighted MDN NLL and adds a penalty for large sigmas.
    
    Args:
        lambda_sigma (float): The strength of the sigma penalty.
    """
    total_loss = 0.0
    num_lines = y_true.shape[1]

    for i in range(num_lines):
        y_target = y_true[:, i:i+1]
        pi, mu, sigma = mdn_params['pi'][i], mdn_params['mu'][i], mdn_params['sigma'][i]
        mask = (y_target != -1).squeeze()

        if mask.sum() == 0:
            continue

        y_target_masked = y_target[mask]
        pi_masked, mu_masked, sigma_masked = pi[mask], mu[mask], sigma[mask]
        
        # --- 1. NLL Loss Calculation (same as before) ---
        dist = Normal(loc=mu_masked, scale=sigma_masked)
        log_prob = dist.log_prob(y_target_masked.expand_as(mu_masked))
        log_mix_prob = torch.log(pi_masked + 1e-8) + log_prob
        log_likelihood = torch.logsumexp(log_mix_prob, dim=1)
        line_nll_loss = -log_likelihood.mean()

        # --- 2. NEW: Sigma Penalty Calculation ---
        # We penalize the mean of the sigmas for the most likely component
        # This focuses the penalty on the component the model actually uses
        most_likely_idx = torch.argmax(pi_masked, dim=1)
        most_likely_sigma = sigma_masked.gather(1, most_likely_idx.unsqueeze(1)).squeeze()
        sigma_penalty = torch.mean(most_likely_sigma)
        
        # --- 3. Combine and Weight ---
        combined_line_loss = line_nll_loss + (lambda_sigma * sigma_penalty)
        total_loss += weights[i] * combined_line_loss

    return total_loss

# In your training_step, you would call this new function:
# loss = weighted_mdn_nll_with_sigma_penalty(y, mdn_params, self.loss_weights, lambda_sigma=0.01)

class CNNLSTM_MDN_MultiHead(pl.LightningModule):
    def __init__(self, input_dim, hidden_dim=128, num_layers=1, hidden_features=64, out_features=32,
                 lr=1e-3, n_components=5, cnn_channels=64, dropout=0.1, num_lines=9):
        super().__init__()
        self.save_hyperparameters()

        # --- Your CNN and LSTM base remains the same ---
        self.fc1 = nn.Linear(input_dim, hidden_features)
        self.ln1 = nn.LayerNorm(hidden_features)
        self.fc2 = nn.Linear(hidden_features, out_features)
        self.ln2 = nn.LayerNorm(out_features)
        self.conv1 = nn.Conv1d(out_features, cnn_channels, kernel_size=1)
        self.bn1 = nn.BatchNorm1d(cnn_channels)
        self.conv3 = nn.Conv1d(out_features, cnn_channels, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm1d(cnn_channels)
        self.mixer = nn.Conv2d(in_channels=2, out_channels=1, kernel_size=1, bias=True)
        fused_dim = cnn_channels
        self.lstm = nn.LSTM(fused_dim, hidden_dim, num_layers=num_layers,
                              batch_first=True, dropout=dropout if num_layers > 1 else 0)

        # === MODIFICATION: Create a list of MDN heads ===
        self.num_lines = num_lines
        self.mdn_heads = nn.ModuleList(
            [nn.Linear(hidden_dim, 3 * n_components) for _ in range(num_lines)]
        )

        self.n_components = n_components
        self.lr = lr

        # === Define importance weights here ===
        # Using exponential decay: w_i = 0.9^(i-1)
        weights = torch.tensor([0.9**i for i in range(self.num_lines)])
        self.register_buffer('loss_weights', weights)

        self.apply(self._init_weights)

    def _init_weights(self, module): # Your init function is fine
        if isinstance(module, nn.Linear):
            nn.init.kaiming_normal_(module.weight, mode="fan_in", nonlinearity="relu")
            if module.bias is not None: nn.init.constant_(module.bias, 0)
        elif isinstance(module, (nn.Conv1d, nn.Conv2d)):
            nn.init.kaiming_normal_(module.weight, mode="fan_out", nonlinearity="relu")
            if module.bias is not None: nn.init.constant_(module.bias, 0)

    def forward(self, X, lengths=None):
        # --- Your forward pass for the base model is the same ---
        x = X["main"]
        x = F.relu(self.ln1(self.fc1(x)))
        x = F.relu(self.ln2(self.fc2(x)))
        x = x.transpose(1, 2)
        x1 = F.relu(self.bn1(self.conv1(x)))
        x3 = F.relu(self.bn3(self.conv3(x)))
        paired = torch.stack([x1, x3], dim=1)
        mixed = self.mixer(paired)
        xf = mixed.squeeze(1).transpose(1, 2)
        
        # We'll assume lengths is None for simplicity here, but your implementation is fine
        _, (h_last, _) = self.lstm(xf)
        last_h = h_last[-1]

        # === MODIFICATION: Get parameters from all heads ===
        all_params = {'pi': [], 'mu': [], 'sigma': []}
        for i in range(self.num_lines):
            raw_params = self.mdn_heads[i](last_h)
            pi, mu, sigma = mdn_split_params(raw_params, self.n_components)
            all_params['pi'].append(pi)
            all_params['mu'].append(mu)
            all_params['sigma'].append(sigma)

        return all_params
    

    # This would be inside your CNNLSTM_MDN_MultiHead class

    def training_step(self, batch, batch_idx):
        # Assuming your batch now provides a y tensor of shape (B, 9)
        # where y has your target line values, padded with -1.
        X, y, lengths = batch

        # Get the dictionary of parameter lists from the forward pass
        mdn_params = self(X, lengths)

        # Calculate loss using the new weighted function
        loss = weighted_mdn_nll_with_sigma_penalty(y, mdn_params, self.loss_weights)

        self.log("train/loss", loss)
        return loss

    def configure_optimizers(self):
        return torch.optim.AdamW(self.parameters(), lr=self.lr)

    # NOTE: You'll also need a validation_step that mirrors the training_step logic
    def validation_step(self, batch, batch_idx):
        X, y, lengths = batch
        mdn_params = self(X, lengths)
        loss = weighted_mdn_nll_with_sigma_penalty(y, mdn_params, self.loss_weights)
        self.log("val/loss", loss, prog_bar=True)
        return loss

## CNNlSTM

In [2]:
import torch
import torch.nn as nn
import pytorch_lightning as pl
import torch.nn.functional as F
from torch.nn.utils.rnn import pack_padded_sequence

def mdn_split_params(raw_params, n_components):
    """
    raw_params: (B, 3K) tensor from mdn_head
    returns:
        pi    (B, K) mixture weights
        mu    (B, K) means
        sigma (B, K) std devs
    """
    B, threeK = raw_params.shape
    assert threeK == 3 * n_components

    raw = raw_params.view(B, n_components, 3)

    pi = raw[..., 0]                 # (B,K)
    mu = raw[..., 1]                 # (B,K)
    sigma = raw[..., 2]              # (B,K)

    pi = F.softmax(pi, dim=-1)       # weights sum to 1
    sigma = F.softplus(sigma) + 1e-4 # strictly positive
    return pi, mu, sigma


def mdn_nll_multitarget(y_line, pi, mu, sigma):
    """
    Negative log-likelihood for MDN with multiple valid targets per sample.
    Args:
        y_line : (B, L) padded targets (0 where invalid)
        pi, mu, sigma : (B, K) MDN params
    Returns:
        scalar loss
    """
    B, K = mu.shape
    losses = []

    for b in range(B):
        valid_y = y_line[b][y_line[b] > 0]  # (M,)
        if len(valid_y) == 0:
            continue

        # expand to (M, K)
        y_exp = valid_y.unsqueeze(-1).expand(-1, K)

        log_prob = -0.5 * ((y_exp - mu[b]) / (sigma[b] + 1e-8))**2 \
                   - torch.log(sigma[b] + 1e-8) \
                   - 0.5 * torch.log(torch.tensor(2.0 * torch.pi, device=y_line.device))

        log_mix = torch.log(pi[b] + 1e-8) + log_prob
        log_sum = torch.logsumexp(log_mix, dim=-1)  # (M,)

        losses.append(-log_sum.mean())

    if len(losses) == 0:
        return torch.tensor(0.0, device=y_line.device, requires_grad=True)

    return torch.stack(losses).mean()


class CNNLSTM_MDN(pl.LightningModule):
    def __init__(self, input_dim, hidden_dim=128, num_layers=1, hidden_features=64, out_features=32,
                 lr=1e-3, n_components=5, cnn_channels=64, dropout=0.1):
        super().__init__()
        self.save_hyperparameters()

        # Time-distributed feature extractor
        self.fc1 = nn.Linear(input_dim, hidden_features)
        self.ln1 = nn.LayerNorm(hidden_features) # ADDED: LayerNorm for time-step features
        self.fc2 = nn.Linear(hidden_features, out_features)
        self.ln2 = nn.LayerNorm(out_features) # ADDED: LayerNorm

        # CNN feature extractors
        self.conv1 = nn.Conv1d(out_features, cnn_channels, kernel_size=1)
        self.bn1 = nn.BatchNorm1d(cnn_channels) # ADDED: BatchNorm for convolutional features
        self.conv3 = nn.Conv1d(out_features, cnn_channels, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm1d(cnn_channels) # ADDED: BatchNorm

        # Learnable mixer for CNN outputs
        self.mixer = nn.Conv2d(in_channels=2, out_channels=1, kernel_size=1, bias=True)

        # LSTM for temporal dependency
        fused_dim = cnn_channels # Input to LSTM is the mixed CNN output
        self.lstm = nn.LSTM(fused_dim, hidden_dim, num_layers=num_layers,
                            batch_first=True, dropout=dropout if num_layers > 1 else 0)

        # MDN Head
        self.mdn_head = nn.Linear(hidden_dim, 3 * n_components)
        self.n_components = n_components
        self.lr = lr

        # Apply weight initialization
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            nn.init.kaiming_normal_(module.weight, mode="fan_in", nonlinearity="relu")
            if module.bias is not None:
                nn.init.constant_(module.bias, 0)
        elif isinstance(module, (nn.Conv1d, nn.Conv2d)):
            nn.init.kaiming_normal_(module.weight, mode="fan_out", nonlinearity="relu")
            if module.bias is not None:
                nn.init.constant_(module.bias, 0)

    def forward(self, X, lengths=None):
        # Input shape X["main"]: (B, T, F_in)
        x = X["main"] # REMOVED redundant transposes

        # 1. Time-distributed feature extraction
        x = self.fc1(x)
        x = F.relu(self.ln1(x)) # CHANGED: Apply LayerNorm before ReLU
        x = self.fc2(x)
        x = F.relu(self.ln2(x)) # CHANGED: Apply LayerNorm before ReLU
        # 2. CNN feature extraction
        x = x.transpose(1, 2)   # Shape: (B, C_in, L=T)
        x1 = F.relu(self.bn1(self.conv1(x))) # CHANGED: Apply BatchNorm before ReLU
        x3 = F.relu(self.bn3(self.conv3(x))) # CHANGED: Apply BatchNorm before ReLU

        # 3. Mix CNN outputs
        paired = torch.stack([x1, x3], dim=1) # Shape: (B, 2, C_out, L)
        mixed = self.mixer(paired)            # Shape: (B, 1, C_out, L)
        
        # Prepare for LSTM
        xf = mixed.squeeze(1).transpose(1, 2) # Shape: (B, L, C_out)

        # 4. LSTM for sequence summary
        if lengths is not None:
            packed_input = pack_padded_sequence(
                xf, lengths.cpu(), batch_first=True, enforce_sorted=False
            )
            _, (h_last, _) = self.lstm(packed_input)
        else:
            _, (h_last, _) = self.lstm(xf)
        
        last_h = h_last[-1] # Shape: (B, H)
        
        # 5. MDN head for distribution parameters
        raw = self.mdn_head(last_h)
        pi, mu, sigma = mdn_split_params(raw, self.n_components)
        return {"pi": pi, "mu": mu, "sigma": sigma}

    def training_step(self, batch, batch_idx):
        X, y_line, lengths = batch
        mdn = self(X, lengths)
        loss = mdn_nll_multitarget(y_line, mdn["pi"], mdn["mu"], mdn["sigma"])
        self.log("train/loss", loss)
        return loss

    def validation_step(self, batch, batch_idx):
        X, y_line, lengths = batch
        mdn = self(X, lengths)
        loss = mdn_nll_multitarget(y_line, mdn["pi"], mdn["mu"], mdn["sigma"])
    # Log everything to progress bar
        self.log("val/loss", loss, prog_bar=True)
        self.log("val/pi_mean", mdn["pi"].mean(), prog_bar=True)
        self.log("val/pi_std", mdn["pi"].std(), prog_bar=True)
        self.log("val/mu_mean", mdn["mu"].mean(), prog_bar=True)
        self.log("val/mu_std", mdn["mu"].std(), prog_bar=True)
        self.log("val/sigma_mean", mdn["sigma"].mean(), prog_bar=True)
        self.log("val/sigma_std", mdn["sigma"].std(), prog_bar=True)
        
    # Inside your CNNLSTM_MDN class
    # def configure_optimizers(self):
    #     optimizer = torch.optim.Adam(self.parameters(), lr=self.lr)
        
    #     # Define the scheduler
    #     scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    #         optimizer,
    #         mode='min',      # We want to minimize the validation loss
    #         factor=0.5,      # Reduce LR by 50% (1.0 -> 0.2)
    #         patience=10,      # Wait 5 validation epochs with no improvement before reducing
    #         verbose=True
    #     )
        
    #     return {
    #         "optimizer": optimizer,
    #         "lr_scheduler": {
    #             "scheduler": scheduler,
    #             "monitor": "val/loss",  # The metric to watch
    #         },
    #     }
    
    def configure_optimizers(self): 
        return torch.optim.Adam(self.parameters(), lr=self.lr)
    # def configure_optimizers(self):
    #     optimizer = torch.optim.Adam(self.parameters(), lr=self.lr)
    #     scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    #         optimizer,
    #         mode="min",
    #         factor=0.2,   # Reduce LR by 80%
    #         patience=5,   # After 5 epochs of no val_loss improvement
    #         verbose=True
    #     )
    #     return {
    #         "optimizer": optimizer,
    #         "lr_scheduler": {
    #             "scheduler": scheduler,
    #             "monitor": "val/loss",  # Important!
    #         },
        # }


## CNNLSTM scalie

In [2]:
import torch
import torch.nn as nn
import pytorch_lightning as pl
import torch.nn.functional as F
from torch.nn.utils.rnn import pack_padded_sequence

def mdn_split_params(raw_params, n_components, mu_scale=10, mu_bias=.9, sigma_scale=10.0):
    """
    Split raw MDN parameters into (pi, mu, sigma).

    Args:
        raw_params: (B, 3 * K) from the network
        n_components: number of mixture components
        mu_scale: scaling factor for mu (default 1.0 = no scaling)
        mu_bias: shift/bias applied after scaling
        sigma_scale: scaling factor for sigma (default 10.0)
    """
    B = raw_params.size(0)
    raw = raw_params.view(B, n_components, 3)

    pi_raw = raw[..., 0]
    mu_raw = raw[..., 1]
    sigma_raw = raw[..., 2]

    pi = F.softmax(pi_raw, dim=-1)
    mu = mu_raw / mu_scale + mu_bias
    sigma = F.softplus(sigma_raw / sigma_scale) + 1e-4

    return pi, mu, sigma


def mdn_nll_multitarget(y_line, pi, mu, sigma):
    """
    Negative log-likelihood for MDN with multiple valid targets per sample.
    Args:
        y_line : (B, L) padded targets (0 where invalid)
        pi, mu, sigma : (B, K) MDN params
    Returns:
        scalar loss
    """
    B, K = mu.shape
    losses = []

    for b in range(B):
        valid_y = y_line[b][y_line[b] > 0]  # (M,)
        if len(valid_y) == 0:
            continue

        # expand to (M, K)
        y_exp = valid_y.unsqueeze(-1).expand(-1, K)

        log_prob = -0.5 * ((y_exp - mu[b]) / (sigma[b] + 1e-8))**2 \
                   - torch.log(sigma[b] + 1e-8) \
                   - 0.5 * torch.log(torch.tensor(2.0 * torch.pi, device=y_line.device))

        log_mix = torch.log(pi[b] + 1e-8) + log_prob
        log_sum = torch.logsumexp(log_mix, dim=-1)  # (M,)

        losses.append(-log_sum.mean())

    if len(losses) == 0:
        return torch.tensor(0.0, device=y_line.device, requires_grad=True)

    return torch.stack(losses).mean()


class CNNLSTM_MDN(pl.LightningModule):
    def __init__(self, input_dim, hidden_dim=128, num_layers=1, hidden_features=64, out_features=32,
                 lr=1e-3, n_components=5, cnn_channels=64, dropout=0.1):
        super().__init__()
        self.save_hyperparameters()

        # Time-distributed feature extractor
        self.fc1 = nn.Linear(input_dim, hidden_features)
        self.ln1 = nn.LayerNorm(hidden_features) # ADDED: LayerNorm for time-step features
        self.fc2 = nn.Linear(hidden_features, out_features)
        self.ln2 = nn.LayerNorm(out_features) # ADDED: LayerNorm

        # CNN feature extractors
        self.conv1 = nn.Conv1d(out_features, cnn_channels, kernel_size=1)
        self.bn1 = nn.BatchNorm1d(cnn_channels) # ADDED: BatchNorm for convolutional features
        self.conv3 = nn.Conv1d(out_features, cnn_channels, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm1d(cnn_channels) # ADDED: BatchNorm

        # Learnable mixer for CNN outputs
        self.mixer = nn.Conv2d(in_channels=2, out_channels=1, kernel_size=1, bias=True)

        # LSTM for temporal dependency
        fused_dim = cnn_channels # Input to LSTM is the mixed CNN output
        self.lstm = nn.LSTM(fused_dim, hidden_dim, num_layers=num_layers,
                            batch_first=True, dropout=dropout if num_layers > 1 else 0)

        # MDN Head
        self.mdn_head = nn.Linear(hidden_dim, 3 * n_components)
        self.n_components = n_components
        self.lr = lr

        # Apply weight initialization
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            nn.init.kaiming_normal_(module.weight, mode="fan_in", nonlinearity="relu")
            if module.bias is not None:
                nn.init.constant_(module.bias, 0)
        elif isinstance(module, (nn.Conv1d, nn.Conv2d)):
            nn.init.kaiming_normal_(module.weight, mode="fan_out", nonlinearity="relu")
            if module.bias is not None:
                nn.init.constant_(module.bias, 0)

    def forward(self, X, lengths=None):
        # Input shape X["main"]: (B, T, F_in)
        x = X["main"] 

        # --- Debug print first candle ---
        # if x.ndim == 3:  # batched: (B, T, F)
        #     first_candle = x[0, 0, :]   # first sample, first time step, all features
        #     print("First candle features:", first_candle.detach().cpu().numpy())
        # elif x.ndim == 2:  # single sequence: (T, F)
        #     first_candle = x[0, :]      # first time step, all features
        #     print("First candle features:", first_candle.detach().cpu().numpy())
        # else:
        #     print("Unexpected shape for x:", x.shape)
        # 1. Time-distributed feature extraction
        x = self.fc1(x)
        x = F.relu(self.ln1(x)) # CHANGED: Apply LayerNorm before ReLU
        x = self.fc2(x)
        x = F.relu(self.ln2(x)) # CHANGED: Apply LayerNorm before ReLU

        # 2. CNN feature extraction
        x = x.transpose(1, 2)   # Shape: (B, C_in, L=T)
        x1 = F.relu(self.bn1(self.conv1(x))) # CHANGED: Apply BatchNorm before ReLU
        x3 = F.relu(self.bn3(self.conv3(x))) # CHANGED: Apply BatchNorm before ReLU

        # 3. Mix CNN outputs
        paired = torch.stack([x1, x3], dim=1) # Shape: (B, 2, C_out, L)
        mixed = self.mixer(paired)            # Shape: (B, 1, C_out, L)
        
        # Prepare for LSTM
        xf = mixed.squeeze(1).transpose(1, 2) # Shape: (B, L, C_out)

        # 4. LSTM for sequence summary
        if lengths is not None:
            packed_input = pack_padded_sequence(
                xf, lengths.cpu(), batch_first=True, enforce_sorted=False
            )
            _, (h_last, _) = self.lstm(packed_input)
        else:
            _, (h_last, _) = self.lstm(xf)
        
        last_h = h_last[-1] # Shape: (B, H)
        
        # 5. MDN head for distribution parameters
        raw = self.mdn_head(last_h)
        pi, mu, sigma = mdn_split_params(raw, self.n_components)
        return {"pi": pi, "mu": mu, "sigma": sigma}

    def training_step(self, batch, batch_idx):
        X, y_line, lengths = batch
        mdn = self(X, lengths)
        loss = mdn_nll_multitarget(y_line, mdn["pi"], mdn["mu"], mdn["sigma"])
        self.log("train/loss", loss)
        return loss

    def validation_step(self, batch, batch_idx):
        X, y_line, lengths = batch
        mdn = self(X, lengths)
        loss = mdn_nll_multitarget(y_line, mdn["pi"], mdn["mu"], mdn["sigma"])
    # Log everything to progress bar
        self.log("val/loss", loss, prog_bar=True)
        self.log("val/pi_mean", mdn["pi"].mean(), prog_bar=True)
        self.log("val/pi_std", mdn["pi"].std(), prog_bar=True)
        self.log("val/mu_mean", mdn["mu"].mean(), prog_bar=True)
        self.log("val/mu_std", mdn["mu"].std(), prog_bar=True)
        self.log("val/sigma_mean", mdn["sigma"].mean(), prog_bar=True)
        self.log("val/sigma_std", mdn["sigma"].std(), prog_bar=True)
        
    # # Inside your CNNLSTM_MDN class
    def configure_optimizers(self):
        return torch.optim.AdamW(self.parameters(), lr=self.lr, weight_decay=1e-4)
    # def configure_optimizers(self):
    #     optimizer = torch.optim.Adam(self.parameters(), lr=self.lr)
        
    #     # Define the scheduler
    #     scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    #         optimizer,
    #         mode='min',      # We want to minimize the validation loss
    #         factor=0.5,      # Reduce LR by 80% (1.0 -> 0.2)
    #         patience=10,      # Wait 5 validation epochs with no improvement before reducing
    #         verbose=True
    #     )
        
    #     return {
    #         "optimizer": optimizer,
    #         "lr_scheduler": {
    #             "scheduler": scheduler,
    #             "monitor": "val/loss",  # The metric to watch
    #         },
    #     }
    # def configure_optimizers(self):
    #     return torch.optim.Adam(self.parameters(), lr=self.hparams.lr)


# data manipulation

In [32]:
import pandas as pd
df_labels = pd.read_csv("/home/iatell/projects/meta-learning/data/line_seq_ordered.csv")
cols = [f'price_line{i}' for i in range(1, 10)]
df_labels = df_labels.dropna(subset=cols, how='all')
df_labels = df_labels.rename(columns={c: c.replace('price_line', 'linePrice_') 
                        for c in df_labels.columns if c.startswith('price_line')})
df_labels.to_csv("/home/iatell/projects/meta-learning/data/line_seq_ordered.csv", index=False)      
#     # overwrites the old file
df_labels

,startTime,endTime,startIndex,endIndex,linePrice_1,linePrice_2,linePrice_3,linePrice_4,linePrice_5,linePrice_6,linePrice_7,linePrice_8,linePrice_9
0,1514764800,1515110400,0,4,NaN,0.878016,0.788209,NaN,NaN,NaN,NaN,NaN,NaN
1,1514764800,1515283200,0,6,NaN,1.055290,0.923251,0.828937,NaN,NaN,NaN,NaN,NaN
2,1515024000,1515369600,3,7,1.143628,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1515456000,1514937600,2,8,1.139775,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1515110400,1515542400,4,9,1.143279,0.964469,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
328,1651795200,1649116800,1555,1586,0.873150,0.825739,0.905267,0.938913,NaN,NaN,NaN,0.955736,NaN
330,1652054400,1652227200,1589,1591,1.063729,NaN,NaN,1.023085,NaN,NaN,NaN,NaN,NaN
331,1652572800,1651881600,1587,1595,0.813907,0.870793,NaN,NaN,NaN,NaN,NaN,0.788406,0.904141
332,1653264000,1652227200,1591,1603,1.042211,1.075683,0.992004,0.958532,NaN,NaN,NaN,NaN,NaN


# train

## simple

In [ ]:
import sys
from pathlib import Path

# Current notebook location
notebook_path = Path().resolve()

# Add parent folder (meta/) to sys.path
sys.path.append(str(notebook_path.parent))
import joblib
import torch
import pytorch_lightning as pl
from torch.utils.data import DataLoader
from datetime import datetime
from preprocess.multi_regression_seq_dif import preprocess_sequences_csv_multilines
# from models.LSTM.lstm_multi_line_reg_seq_dif import LSTMMultiRegressor
from utils.make_step import make_step
from utils.padding_batch_reg import collate_batch
from utils.get_init_argumens import get_init_args
import pandas as pd
import io
import numpy as np
import os
from add_ons.drop_column import drop_columns
from add_ons.normalize_candle_seq import add_label_normalized_candles
from add_ons.feature_pipeline3 import FeaturePipeline
from add_ons.candle_dif_rate_of_change_percentage import add_candle_rocp
from add_ons.candle_rate_of_change import add_candle_ratios
from sklearn.metrics import accuracy_score, f1_score
# ---------------- Evaluation ---------------- #
@torch.no_grad()
def evaluate_model_mdn(model, val_loader, zero_idx=0, threshold=0.1):
    """
    Evaluate CNN–LSTM–MDN model (last-output version).

    Args
    ----
    model : pl.LightningModule with MDN forward
    val_loader : DataLoader yielding (X, y, lengths)
    zero_idx : which mixture component is considered "no-line" (usually 0)
    threshold : if pi[:,zero_idx] > threshold → predict invalid

    Returns
    -------
    dict with mse, mae, acc, f1
    """
    model.eval()
    all_preds_reg, all_labels_reg = [], []
    all_preds_len, all_labels_len = [], []

    device = next(model.parameters()).device

    with torch.no_grad():
        for X_batch, y_batch, lengths in val_loader:
            if isinstance(X_batch, dict):
                X_batch = {k: v.to(device) for k, v in X_batch.items()}
            else:
                X_batch = X_batch.to(device)

            y_batch = y_batch.to(device)
            mdn = model(X_batch, lengths)
            pi, mu, sigma = mdn["pi"], mdn["mu"], mdn["sigma"]  # (B,K)

            # regression expectation
            y_pred = (pi * mu).sum(dim=-1)  # (B,)
            B = y_batch.size(0)
            y_len = (y_batch > 0).sum(dim=1)                # (B,)
            idx = torch.clamp(y_len - 1, min=0)             # last valid index
            y_true = y_batch[torch.arange(B, device=y_batch.device), idx]  # (B,)
            # only last step
            # print("lengths(features):", lengths[:10])
            # print("lengths(labels):", y_len[:10])

            all_preds_reg.append(y_pred.cpu().numpy())
            all_labels_reg.append(y_true.cpu().numpy())

            # validity classification
            pi_zero = pi[:, zero_idx]  # (B,)
            pred_valid = (pi_zero < (1 - threshold)).long()
            true_valid = torch.ones_like(pred_valid)  # last step always valid

            all_preds_len.extend(pred_valid.cpu().numpy().tolist())
            all_labels_len.extend(true_valid.cpu().numpy().tolist())


        # ----- Regression metrics -----
    all_preds_reg = np.concatenate(all_preds_reg)  # (N,)
    all_labels_reg = np.concatenate(all_labels_reg)
    mse = ((all_preds_reg - all_labels_reg) ** 2).mean()
    mae = np.abs(all_preds_reg - all_labels_reg).mean()
    # ----- Validity metrics -----
    acc = accuracy_score(all_labels_len, all_preds_len)
    f1 = f1_score(all_labels_len, all_preds_len, average="macro")

    print("\n📊 Validation Metrics (MDN, last-output):")
    print(f"  Regression → MSE: {mse:.6f}, MAE: {mae:.6f}")
    print(f"  Validity   → Acc: {acc:.4f}, F1: {f1:.4f}")

    return {"mse": mse, "mae": mae, "acc": acc, "f1": f1}
# ---------------- Train ---------------- #
def train_model(
    data_csv,
    labels_csv,
    model_out_dir="models/saved_models",
    do_validation=True,
    hidden_dim=200,
    num_layers=1,
    lr=0.001,
    batch_size=32,
    max_epochs=1000,
    save_model=False,
    return_val_accuracy = True,
    test_mode = False,
    early_stop = False
):

    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    model_out = f"{model_out_dir}/lstm_model_multireg_{timestamp}.pt"
    meta_out  = f"{model_out_dir}/lstm_meta_multireg_{timestamp}.pkl"

    pipeline = FeaturePipeline(
        steps=[
            # make_step(add_label_normalized_candles),
            make_step(add_candle_rocp),
            make_step(drop_columns, cols_to_drop=["open","high","low","close","volume"]),
            
        ],
        # norm_methods={
        #     "main": {
        #         "upper_shadow": "robust", "body": "standard", "lower_shadow": "standard",
        #         "upper_body_ratio": "standard", "lower_body_ratio": "standard",
        #         "upper_lower_body_ratio": "standard", "Candle_Color": "standard"
        #     }
        # },
        per_window_flags=[
            False, 
          False, 
        #   True
                ]
    )
    # Preprocess: pad linePrices and sequences
    if do_validation:
        train_ds, val_ds, df, feature_cols, max_len_y = preprocess_sequences_csv_multilines(
            data_csv, labels_csv,
            val_split=True,
            for_xgboost=False,
            debug_sample=True,
            feature_pipeline=pipeline
        )
    else:
        train_ds, df, feature_cols, max_len_y = preprocess_sequences_csv_multilines(
            data_csv, labels_csv,
            val_split=False,
            for_xgboost=False,
            debug_sample=False,
            feature_pipeline=pipeline
        )
        val_ds = None

    sample = train_ds[0][0]  # first sample's features
    if isinstance(sample, dict):  # multiple feature groups
        input_dim = sample['main'].shape[1]
    else:  # single tensor
        input_dim = sample.shape[1]

    model = CNNLSTM_MDN(
        input_dim=input_dim,
        hidden_dim=hidden_dim,
        num_layers=num_layers,
        lr=lr
    )
    init_args = get_init_args(model, input_dim=input_dim, hidden_dim=hidden_dim, num_layers=num_layers, lr=lr)

    model_class_info = {
        "module": model.__class__.__module__,
        "class": model.__class__.__name__,
        "init_args": init_args
    }

    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, collate_fn=collate_batch)
    val_loader = DataLoader(val_ds, batch_size=batch_size, collate_fn=collate_batch) if val_ds else None
    
    # --- Debug / Test mode --- #
    if test_mode:
        save_model = False
        from itertools import islice

        # Try to grab 3rd batch; if not available, take first
        try:
            batch = next(islice(iter(train_loader), 2, 3))
        except StopIteration:
            batch = next(iter(train_loader))

        X_batch_dict, y_batch, lengths = batch

        print("🔍 Debug batch:")
        if isinstance(X_batch_dict, dict):
            print("  Keys in X_batch:", list(X_batch_dict.keys()))
        print("  y_batch shape:", y_batch.shape)
        print("  First label in batch:", y_batch[0])

        # --- Track real column names for each feature group ---
        feature_names_dict = {}
        for name, X_batch in X_batch_dict.items():
            if name == "main":
                # Use actual feature columns after preprocessing
                feature_names_dict[name] = feature_cols
            else:
                # For extra feature groups, fallback to generic names
                feature_names_dict[name] = [f"{name}_{i}" for i in range(X_batch.shape[2])]

        dfs = []
        for name, X_batch in X_batch_dict.items():
            print(f"\nFeature group: {name}")
            print("  X_batch shape:", X_batch.shape)
            print("  First sequence in batch (first  steps):\n", X_batch[0][:])

            batch_size_, seq_len, feature_dim = X_batch.shape
            df_part = pd.DataFrame(
                X_batch.reshape(batch_size_ * seq_len, feature_dim).numpy(),
                columns=feature_names_dict[name]
            )
            dfs.append(df_part)

        # Combine all feature groups horizontally
        global df_seq
        df_seq = pd.concat(dfs, axis=1)
        print("\n✅ Combined df_seq shape:", df_seq.shape)
        print("✅ Column names in df_seq:", df_seq.columns.tolist())

    # --- Early stopping --- #
    if early_stop == True:
        from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint
        early_stop_callback = EarlyStopping(
            monitor="val_loss",   # metric to monitor (must be logged in your LightningModule)
            patience=10,          # number of epochs with no improvement before stopping
            min_delta=0.001,      # minimum improvement to qualify as "better"
            mode="min",           # "min" for loss, "max" for accuracy
            verbose=True
        )

        checkpoint_callback = ModelCheckpoint(
            dirpath=model_out_dir,
            filename="best_model",
            save_top_k=1,
            monitor="val_loss",
            mode="min"
        )
        callbacks=[early_stop_callback,checkpoint_callback]

    trainer = pl.Trainer(
        max_epochs=max_epochs,
        accelerator="auto",
        devices=1,
        fast_dev_run=test_mode,
        gradient_clip_val=1.0,
        gradient_clip_algorithm="norm",
        callbacks= callbacks if early_stop else None
    )

    trainer.fit(model, train_loader, val_loader)

    if save_model:
        os.makedirs(model_out_dir, exist_ok=True)
        trainer.save_checkpoint(model_out)
        joblib.dump({
    "input_dim": input_dim,
    "hidden_dim": hidden_dim,
    "num_layers": num_layers,
    "max_len_y": max_len_y,
    "feature_cols": feature_cols,
    "scalers": pipeline.scalers,
    "pipeline_config": pipeline.export_config(),
    "model_class_info": model_class_info   # ✅ save model class info
}, meta_out)
        
    # --- Evaluation --- #
    if do_validation:
        mse, mae, acc, f1 = evaluate_model_mdn(model, val_loader)
        if return_val_accuracy:
            return {"mse": mse, "mae": mae, "acc": acc, "f1": f1}
        
if __name__ == "__main__":
    train_model(
        "/home/iatell/projects/meta-learning/data/Bitcoin_BTCUSDT_kaggle_1D_candles.csv",
        "/home/iatell/projects/meta-learning/data/seq_line_labels.csv",
        save_model=True,
        do_validation=True,
        test_mode = False
    )


## ordered

In [ ]:
import sys
from pathlib import Path

# Current notebook location
notebook_path = Path().resolve()

# Add parent folder (meta/) to sys.path
sys.path.append(str(notebook_path.parent))
import joblib
import torch
import pytorch_lightning as pl
from torch.utils.data import DataLoader
from datetime import datetime
from preprocess.multi_regression_seq_dif import preprocess_sequences_csv_multilines
# from models.LSTM.lstm_multi_line_reg_seq_dif import LSTMMultiRegressor
from utils.make_step import make_step
from utils.padding_batch_reg import collate_batch
from utils.get_init_argumens import get_init_args
import pandas as pd
import io
import numpy as np
import os
from add_ons.drop_column import drop_columns
from add_ons.normalize_candle_seq import add_label_normalized_candles
from add_ons.feature_pipeline3 import FeaturePipeline
from add_ons.candle_dif_rate_of_change_percentage import add_candle_rocp
from add_ons.candle_rate_of_change import add_candle_ratios
from sklearn.metrics import accuracy_score, f1_score,mean_squared_error,mean_absolute_error
from utils.to_address import to_address
# ---------------- Evaluation ---------------- #
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, accuracy_score, f1_score
import torch

@torch.no_grad()
def evaluate_model_mdn(model, val_loader, threshold=0.1):
    """
    Evaluate CNN–LSTM–MDN model (multi-head, top-pi selection per line).

    Args
    ----
    model : pl.LightningModule with multi-head MDN forward
    val_loader : DataLoader yielding (X, y, lengths)
    threshold : optional threshold for validity classification

    Returns
    -------
    dict with mse, mae, acc, f1
    """
    model.eval()
    all_preds_reg, all_labels_reg = [], []
    all_preds_len, all_labels_len = [], []

    device = next(model.parameters()).device

    for X_batch, y_batch, lengths in val_loader:
        # Move to device
        if isinstance(X_batch, dict):
            X_batch = {k: v.to(device) for k, v in X_batch.items()}
        else:
            X_batch = X_batch.to(device)
        y_batch = y_batch.to(device)

        # Forward pass
        mdn_params = model(X_batch, lengths)

        B, num_lines = y_batch.shape
        y_pred_lines = []

        for i in range(num_lines):
            pi, mu = mdn_params['pi'], mdn_params['mu']  # both (B, n_components)
            
            # Pick component with highest pi per sample
            top_idx = torch.argmax(pi, dim=1, keepdim=True)     # (B,1)
            selected_mu = mu.gather(1, top_idx).squeeze(1)     # (B,)

            # Mask padded targets
            mask = (y_batch[:, i] != 0)
            selected_mu[~mask] = 0.0

            y_pred_lines.append(selected_mu)

        y_pred_all = torch.stack(y_pred_lines, dim=1)  # (B, num_lines)

        # Last valid step per sample
        y_len = (y_batch > 0).sum(dim=1)
        idx = torch.clamp(y_len - 1, min=0)
        y_true = y_batch[torch.arange(B), idx]
        y_pred = y_pred_all[torch.arange(B), idx]

        all_preds_reg.append(y_pred.cpu().numpy())
        all_labels_reg.append(y_true.cpu().numpy())

        # --- Validity classification ---
        pred_valid_lines = []
        for i in range(num_lines):
            pi = mdn_params['pi']    # (B, n_components)
            top_idx = torch.argmax(pi, dim=1, keepdim=True)
            pi_max = pi.gather(1, top_idx).squeeze(1)
            pred_valid_lines.append((pi_max > threshold).long())

        pred_valid_all = torch.stack(pred_valid_lines, dim=1)
        pred_valid_last = pred_valid_all[torch.arange(B), idx]
        true_valid_last = torch.ones_like(pred_valid_last)

        all_preds_len.extend(pred_valid_last.cpu().numpy().tolist())
        all_labels_len.extend(true_valid_last.cpu().numpy().tolist())

    # Concatenate all batches
    y_pred_reg = np.concatenate(all_preds_reg)
    y_true_reg = np.concatenate(all_labels_reg)

    mse = mean_squared_error(y_true_reg, y_pred_reg)
    mae = mean_absolute_error(y_true_reg, y_pred_reg)
    acc = accuracy_score(all_labels_len, all_preds_len)
    f1 = f1_score(all_labels_len, all_preds_len)

    print("mse:", mse, "mae:", mae, "acc:", acc, "f1:", f1)
    return {"mse": mse, "mae": mae, "acc": acc, "f1": f1}

# ---------------- Train ---------------- #
def train_model(
    data_csv,
    labels_csv,
    model_out_dir="models/saved_models",
    do_validation=True,
    hidden_dim=200,
    num_layers=1,
    lr=0.001,
    batch_size=32,
    max_epochs=1000,
    save_model=False,
    return_val_accuracy = True,
    test_mode = False,
    early_stop = False
):

    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    model_out = f"{model_out_dir}/lstm_model_multireg_{timestamp}.pt"
    meta_out  = f"{model_out_dir}/lstm_meta_multireg_{timestamp}.pkl"

    pipeline = FeaturePipeline(
        steps=[
            # make_step(add_label_normalized_candles),
            make_step(add_candle_rocp),
            make_step(drop_columns, cols_to_drop=["open","high","low","close","volume"]),
            
        ],
        # norm_methods={
        #     "main": {
        #         "upper_shadow": "robust", "body": "standard", "lower_shadow": "standard",
        #         "upper_body_ratio": "standard", "lower_body_ratio": "standard",
        #         "upper_lower_body_ratio": "standard", "Candle_Color": "standard"
        #     }
        # },
        per_window_flags=[
            False, 
          False, 
        #   True
                ]
    )
    # Preprocess: pad linePrices and sequences

    if do_validation:
        train_ds, val_ds, df, feature_cols, max_len_y = preprocess_sequences_csv_multilines(
            data_csv, labels_csv,
            val_split=True,
            for_xgboost=False,
            debug_sample=True,
            feature_pipeline=pipeline,
            preserve_order= True
        )
    else:
        train_ds, df, feature_cols, max_len_y = preprocess_sequences_csv_multilines(
            data_csv, labels_csv,
            val_split=False,
            for_xgboost=False,
            debug_sample=False,
            feature_pipeline=pipeline,
            preserve_order= True
        )
        val_ds = None

    sample = train_ds[0][0]  # first sample's features
    if isinstance(sample, dict):  # multiple feature groups
        input_dim = sample['main'].shape[1]
    else:  # single tensor
        input_dim = sample.shape[1]

    model = CNNLSTM_MDN(
        input_dim=input_dim,
        hidden_dim=hidden_dim,
        num_lines=max_len_y,
        lr=lr
    )
    init_args = get_init_args(model, input_dim=input_dim, hidden_dim=hidden_dim, num_layers=num_layers, lr=lr)

    model_class_info = {
        "module": model.__class__.__module__,
        "class": model.__class__.__name__,
        "init_args": init_args
    }

    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, collate_fn=collate_batch)
    val_loader = DataLoader(val_ds, batch_size=batch_size, collate_fn=collate_batch) if val_ds else None
    
    # --- Debug / Test mode --- #
    if test_mode:
        save_model = False
        from itertools import islice

        # Try to grab 3rd batch; if not available, take first
        try:
            batch = next(islice(iter(train_loader), 2, 3))
        except StopIteration:
            batch = next(iter(train_loader))

        X_batch_dict, y_batch, lengths = batch

        print("🔍 Debug batch:")
        if isinstance(X_batch_dict, dict):
            print("  Keys in X_batch:", list(X_batch_dict.keys()))
        print("  y_batch shape:", y_batch.shape)
        print("  First label in batch:", y_batch[0])

        # --- Track real column names for each feature group ---
        feature_names_dict = {}
        for name, X_batch in X_batch_dict.items():
            if name == "main":
                # Use actual feature columns after preprocessing
                feature_names_dict[name] = feature_cols
            else:
                # For extra feature groups, fallback to generic names
                feature_names_dict[name] = [f"{name}_{i}" for i in range(X_batch.shape[2])]

        dfs = []
        for name, X_batch in X_batch_dict.items():
            print(f"\nFeature group: {name}")
            print("  X_batch shape:", X_batch.shape)
            print("  First sequence in batch (first  steps):\n", X_batch[0][:])

            batch_size_, seq_len, feature_dim = X_batch.shape
            df_part = pd.DataFrame(
                X_batch.reshape(batch_size_ * seq_len, feature_dim).numpy(),
                columns=feature_names_dict[name]
            )
            dfs.append(df_part)

        # Combine all feature groups horizontally
        global df_seq
        df_seq = pd.concat(dfs, axis=1)
        print("\n✅ Combined df_seq shape:", df_seq.shape)
        print("✅ Column names in df_seq:", df_seq.columns.tolist())

    # --- Early stopping --- #
    if early_stop == True:
        from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint
        early_stop_callback = EarlyStopping(
            monitor="val_loss",   # metric to monitor (must be logged in your LightningModule)
            patience=10,          # number of epochs with no improvement before stopping
            min_delta=0.001,      # minimum improvement to qualify as "better"
            mode="min",           # "min" for loss, "max" for accuracy
            verbose=True
        )

        checkpoint_callback = ModelCheckpoint(
            dirpath=model_out_dir,
            filename="best_model",
            save_top_k=1,
            monitor="val_loss",
            mode="min"
        )
        callbacks=[early_stop_callback,checkpoint_callback]

    trainer = pl.Trainer(
        max_epochs=max_epochs,
        accelerator="auto",
        devices=1,
        fast_dev_run=test_mode,
        gradient_clip_val=1.0,
        gradient_clip_algorithm="norm",
        callbacks= callbacks if early_stop else None
    )

    trainer.fit(model, train_loader, val_loader)

    if save_model:
        os.makedirs(model_out_dir, exist_ok=True)
        trainer.save_checkpoint(model_out)
        joblib.dump({
    "input_dim": input_dim,
    "hidden_dim": hidden_dim,
    "num_layers": num_layers,
    "max_len_y": max_len_y,
    "feature_cols": feature_cols,
    "scalers": pipeline.scalers,
    "pipeline_config": pipeline.export_config(),
    "model_class_info": model_class_info   # ✅ save model class info
}, meta_out)
        
    # --- Evaluation --- #
    if do_validation:
        mse, mae, acc, f1 = evaluate_model_mdn(model, val_loader)
        if return_val_accuracy:
            return {"mse": mse, "mae": mae, "acc": acc, "f1": f1}
        
if __name__ == "__main__":
    train_model(
        "/home/iatell/projects/meta-learning/data/Bitcoin_BTCUSDT_kaggle_1D_candles.csv",
        "/home/iatell/projects/meta-learning/data/line_seq_ordered.csv",
        save_model=True,
        do_validation=True,
        test_mode = True
    )


## two head

In [13]:
import torch
import pytorch_lightning as pl
from torch.utils.data import DataLoader
from sklearn.metrics import classification_report, confusion_matrix
from datetime import datetime
from preprocess.multi_regression_seq_dif import preprocess_sequences_csv_multilines
# from models.LSTM.lstm_multi_line_reg_seq_dif import LSTMMultiRegressor
from utils.print_batch import print_batch
from utils.to_address import to_address
from utils.json_to_csv import json_to_csv_in_memory
from utils.padding_batch_reg import collate_batch
import pandas as pd
import io
import numpy as np
import os
from sklearn.metrics import accuracy_score, f1_score
from add_ons.feature_pipeline3 import FeaturePipeline
from add_ons.drop_column import drop_columns
from add_ons.candle_dif_rate_of_change_percentage import add_candle_rocp
from add_ons.candle_proportion import add_candle_proportions
from add_ons.candle_rate_of_change import add_candle_ratios
from utils.make_step import make_step
# ---------------- Evaluation ---------------- #
def evaluate_model(model, val_loader, threshold=0.5):
    model.eval()
    all_preds_reg, all_labels_reg = [], []
    all_preds_len, all_labels_len = [], []

    with torch.no_grad():
        for X_batch, y_batch, lengths in val_loader:
            # Send to same device as model
            device = next(model.parameters()).device
            X_batch = {k: v.to(device) for k, v in X_batch.items()}
            y_batch = y_batch.to(device)
            lengths = lengths.to(device)

            # Forward pass: regression + length logits
            y_pred, len_logits = model(X_batch, lengths)

            # Regression targets
            all_preds_reg.append(y_pred.cpu().numpy())
            all_labels_reg.append(y_batch.cpu().numpy())

            # Length targets
            true_lengths = lengths.cpu().numpy()
            pred_lengths = model.predict_length(len_logits).cpu().numpy()

            all_labels_len.extend(true_lengths.tolist())
            all_preds_len.extend(pred_lengths.tolist())

    # ----- Regression metrics -----
    all_preds_reg = np.vstack(all_preds_reg)
    all_labels_reg = np.vstack(all_labels_reg)

    mse = ((all_preds_reg - all_labels_reg) ** 2).mean()
    mae = np.abs(all_preds_reg - all_labels_reg).mean()

    # ----- Length metrics -----


    acc = accuracy_score(all_labels_len, all_preds_len)
    f1 = f1_score(all_labels_len, all_preds_len, average="macro")

    print("\n📊 Validation Metrics:")
    print(f"  Regression → MSE: {mse:.6f}, MAE: {mae:.6f}")
    print(f"  Length     → Acc: {acc:.4f}, F1: {f1:.4f}")

    return {"mse": mse, "mae": mae, "acc": acc, "f1": f1}


# ---------------- Train ---------------- #
def train_model(
    data_csv,
    labels_csv,
    model_out_dir="models/saved_models",
    do_validation=True,
    hidden_dim=128,
    num_layers=1,
    lr=0.001,
    batch_size=32,
    max_epochs=50,
    save_model=False,
    return_val_accuracy = True,
    test_mode = True,
    early_stop = False
):

    pipeline = FeaturePipeline(
        steps=[
            # make_step(add_label_normalized_candles),
            make_step(add_candle_rocp),
            make_step(drop_columns, cols_to_drop=["open","high","low","close","volume"]),
            
        ],
        # norm_methods={
        #     "main": {
        #         "upper_shadow": "robust", "body": "standard", "lower_shadow": "standard",
        #         "upper_body_ratio": "standard", "lower_body_ratio": "standard",
        #         "upper_lower_body_ratio": "standard", "Candle_Color": "standard"
        #     }
        # },
        per_window_flags=[
            False, 
          False, 
        #   True
                ]
    )
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    model_out = f"{model_out_dir}/lstm_model_multireg_{timestamp}.pt"
    meta_out  = f"{model_out_dir}/lstm_meta_multireg_{timestamp}.pkl"

    # Preprocess: pad linePrices and sequences
    if do_validation:
        train_ds, val_ds, df, feature_cols, max_len_y = preprocess_sequences_csv_multilines(
            data_csv, labels_csv,
            val_split=True,
            for_xgboost=False,
            debug_sample=True,
            feature_pipeline=pipeline
        )
    else:
        train_ds, df, feature_cols, max_len_y = preprocess_sequences_csv_multilines(
            data_csv, labels_csv,
            val_split=False,
            for_xgboost=False,
            debug_sample=False
        )
        val_ds = None

    sample = train_ds[0][0]  # first sample's features
    if isinstance(sample, dict):  # multiple feature groups
        input_dim = sample['main'].shape[1]
    else:  # single tensor
        input_dim = sample.shape[1]

    model = LSTMMultiRegressor(
        input_dim=input_dim,
        hidden_dim=hidden_dim,
        num_layers=num_layers,
        max_len_y=max_len_y,
        lr=lr
    )
    init_args = get_init_args(model, input_dim=input_dim, hidden_dim=hidden_dim, num_layers=num_layers, lr=lr,max_len_y=max_len_y)

    model_class_info = {
        "module": model.__class__.__module__,
        "class": model.__class__.__name__,
        "init_args": init_args
    }

    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, collate_fn=collate_batch)
    val_loader = DataLoader(val_ds, batch_size=batch_size, collate_fn=collate_batch) if val_ds else None
    # --- Early stopping --- #
    if early_stop == True:
        from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint
        early_stop_callback = EarlyStopping(
            monitor="val_loss",   # metric to monitor (must be logged in your LightningModule)
            patience=10,          # number of epochs with no improvement before stopping
            min_delta=0.001,      # minimum improvement to qualify as "better"
            mode="min",           # "min" for loss, "max" for accuracy
            verbose=True
        )

        checkpoint_callback = ModelCheckpoint(
            dirpath=model_out_dir,
            filename="best_model",
            save_top_k=1,
            monitor="val_loss",
            mode="min"
        )
        callbacks=[early_stop_callback,checkpoint_callback]

    trainer = pl.Trainer(
        max_epochs=max_epochs,
        accelerator="auto",
        devices=1,
        fast_dev_run=test_mode,
        gradient_clip_val=1.0,
        gradient_clip_algorithm="norm",
        callbacks= callbacks if early_stop else None
    )

    trainer.fit(model, train_loader, val_loader)

    # --- Debug / Test mode --- #
    if test_mode:
        save_model = False
        from itertools import islice

        # Try to grab 3rd batch; if not available, take first
        try:
            batch = next(islice(iter(train_loader), 2, 3))
        except StopIteration:
            batch = next(iter(train_loader))

        X_batch_dict, y_batch, lengths = batch

        print("🔍 Debug batch:")
        if isinstance(X_batch_dict, dict):
            print("  Keys in X_batch:", list(X_batch_dict.keys()))
        print("  y_batch shape:", y_batch.shape)
        print("  First label in batch:", y_batch[0])

        # --- Track real column names for each feature group ---
        feature_names_dict = {}
        for name, X_batch in X_batch_dict.items():
            if name == "main":
                # Use actual feature columns after preprocessing
                feature_names_dict[name] = feature_cols
            else:
                # For extra feature groups, fallback to generic names
                feature_names_dict[name] = [f"{name}_{i}" for i in range(X_batch.shape[2])]

        dfs = []
        for name, X_batch in X_batch_dict.items():
            print(f"\nFeature group: {name}")
            print("  X_batch shape:", X_batch.shape)
            print("  First sequence in batch (first  steps):\n", X_batch[0][:])

            batch_size_, seq_len, feature_dim = X_batch.shape
            df_part = pd.DataFrame(
                X_batch.reshape(batch_size_ * seq_len, feature_dim).numpy(),
                columns=feature_names_dict[name]
            )
            dfs.append(df_part)

        # Combine all feature groups horizontally
        global df_seq
        df_seq = pd.concat(dfs, axis=1)
        print("\n✅ Combined df_seq shape:", df_seq.shape)
        print("✅ Column names in df_seq:", df_seq.columns.tolist())

        
    if save_model:
        os.makedirs(model_out_dir, exist_ok=True)
        trainer.save_checkpoint(model_out)
        joblib.dump({
            "input_dim": input_dim,
            "hidden_dim": hidden_dim,
            "num_layers": num_layers,
            "max_len_y": max_len_y,
            "feature_cols": feature_cols,
            "scalers": pipeline.scalers,
            "pipeline_config": pipeline.export_config(),
            "model_class_info": model_class_info 
        }, meta_out)
        print(f"✅ Model saved to {model_out}")
        print(f"✅ Meta saved to {meta_out}")


        
    # --- Evaluation --- #
    if do_validation:
        mse, mae, acc, f1 = evaluate_model(model, val_loader)
        if return_val_accuracy:
            return {"mse": mse, "mae": mae, "acc": acc, "f1": f1}
        
if __name__ == "__main__":
    train_model(
        "/home/iatell/projects/meta-learning/data/Bitcoin_BTCUSDT_kaggle_1D_candles.csv",
        "/home/iatell/projects/meta-learning/data/seq_line_labels.csv",
        do_validation=True,
        test_mode = True
    )


💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Running in `fast_dev_run` mode: will run the requested loop using 1 batch(es). Logging and checkpointing is suppressed.
/home/iatell/envs/Rllib2.43/lib/python3.11/site-packages/pytorch_lightning/trainer/configuration_validator.py:68: You passed in a `val_dataloader` but have no `validation_step`. Skipping val loop.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name        | Type              | Params | Mode 
----------------------------------------------------------
0 | lstm        | LSTM              | 68.6 K | train
1 | fc_reg      | Linear            | 774    | train
2 | fc_len      | Linear            | 774    | train
3 | loss_fn_reg | MSELoss           | 0      | t


=== DEBUG SAMPLE CHECK ===

--- Sequence 0 ---
Label: [0.774234 0.941543 1.004005 0.       0.       0.      ] Encoded: [0.774234 0.941543 1.004005 0.       0.       0.      ]
Shape: (6, 4)
First few rows:
 [[ 0.          0.          0.          0.        ]
 [-0.02376365  0.1197622   0.01098196  0.09679447]
 [ 0.09773012 -0.0107235   0.09774849  0.01665405]
 [ 0.01562355 -0.00180042 -0.01639293  0.0093857 ]
 [ 0.00938704  0.12409948  0.04899828  0.12622231]]



/home/iatell/envs/Rllib2.43/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=19` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_steps=1` reached.


🔍 Debug batch:
  Keys in X_batch: ['main']
  y_batch shape: torch.Size([32, 6])
  First label in batch: tensor([0.9305, 0.9831, 1.0201, 0.9013, 0.9481, 0.0000])

Feature group: main
  X_batch shape: torch.Size([32, 53, 4])
  First sequence in batch (first  steps):
 tensor([[-4.1796e-03,  5.3411e-03,  9.4225e-03,  5.2775e-03],
        [ 5.4519e-03,  8.8342e-02,  9.4162e-03,  8.6850e-02],
        [ 8.6649e-02,  2.6106e-02,  6.4823e-02,  4.1008e-02],
        [ 4.1028e-02,  6.3649e-02,  5.7843e-02,  2.5294e-03],
        [ 1.9190e-03,  5.2973e-02,  3.4368e-03,  1.0578e-01],
        [ 1.0706e-01,  4.8519e-03,  3.5488e-02, -5.2094e-02],
        [-5.2225e-02, -3.8887e-02,  5.5792e-03,  2.8721e-02],
        [ 2.8295e-02,  1.0689e-02,  2.2457e-02, -4.8621e-03],
        [-4.8621e-03,  4.3881e-02,  1.4168e-02,  1.9925e-02],
        [ 2.0460e-02, -4.2401e-02, -7.0938e-02, -6.5424e-02],
        [-6.5971e-02, -4.5485e-02,  2.8329e-03,  8.5296e-03],
        [ 8.5915e-03, -6.0383e-03, -3.3292e-02, -5.5

In [5]:
df_seq = df_seq.loc[~(df_seq==0).all(axis=1)]
df_seq

,open_dif,high_dif,low_dif,close_dif
0,-0.002524,0.089635,-0.004985,0.076695
1,0.077057,-0.014345,0.074718,0.001678
2,0.001341,0.001351,-0.004240,0.004060
3,0.003975,-0.000024,0.001598,-0.013318
4,-0.013235,-0.004695,-0.009180,0.000113
...,...,...,...,...
1691,-0.020200,-0.011237,-0.006991,0.003566
1692,0.003566,-0.003089,-0.002702,-0.016342
1693,-0.016342,0.022225,-0.014226,-0.023502
1694,-0.023502,-0.055281,-0.126078,-0.104118


# server

In [ ]:
import sys
from pathlib import Path

# Current notebook location
notebook_path = Path().resolve()

# Add parent folder (meta/) to sys.path
sys.path.append(str(notebook_path.parent))
import glob
import joblib
import torch
import numpy as np
import pandas as pd
from flask import Flask, request, jsonify, render_template
from servers.pre_process.multi_reg_dif_seq import ServerPreprocess, import_class, build_pipeline_from_config
# from models.LSTM.cnn_lstm_mdn import CNNLSTM_MDN  # <-- your updated "last-output" model

app = Flask(__name__)

# ---------------- Load model and meta ----------------
meta_path = glob.glob("/home/iatell/projects/meta-learning/play_grounds/models/saved_models/lstm_meta_multireg_*.pkl")[0]
state_path = glob.glob("/home/iatell/projects/meta-learning/play_grounds/models/saved_models/lstm_model_multireg*.pt")[0]

meta = joblib.load(meta_path)
FEATURES = meta['feature_cols']
print("features",FEATURES)
# ---------------- Model ----------------
# Reconstruct model class
#for python file:
# model_cls_info = meta["model_class_info"]
# ModelClass = import_class(model_cls_info["module"], model_cls_info["class"])
model_cls_info = meta["model_class_info"]
ModelClass = CNNLSTM_MDN
# Initialize model with original args
model = ModelClass(**model_cls_info["init_args"])
model = CNNLSTM_MDN.load_from_checkpoint(state_path)
model.eval()

# ---------------- Load data ----------------
df = pd.read_csv( "/home/iatell/projects/meta-learning/data/Bitcoin_BTCUSDT_kaggle_1D_candles.csv", parse_dates=['timestamp'])

# ---------------- Setup pipeline ----------------
pipeline = build_pipeline_from_config(meta["pipeline_config"])
pipeline.scalers = meta["scalers"]

# Stateful preprocessing instance
preproc = ServerPreprocess(feature_pipeline=pipeline)


# ---------------- Routes ----------------
@app.route("/")
def home():
    return render_template("sequential.html")


@app.route("/get_and_add_data")
def get_and_add_data():
    dense = df.set_index('timestamp').asfreq('D').ffill()
    initial_seq_len = 21
    next_idx = request.args.get("idx", type=int)
    if next_idx is None:
        # First call → load initial candles
        if len(preproc.dataset) == 0:
            for _, row in dense.iloc[:initial_seq_len].iterrows():
                preproc.add_candle(row)
        candles = [
            {'time': int(ts.timestamp()),
             'open': float(row.open),
             'high': float(row.high),
             'low': float(row.low),
             'close': float(row.close)}
            for ts, row in dense.iloc[:initial_seq_len].iterrows()
        ]
        print("Returning initial candles:", candles)

        return jsonify({
            "initial_seq_len": initial_seq_len,
            "next_idx": initial_seq_len,
            "candles": candles
        })
    else:
        # Subsequent calls → 1 candle
        if next_idx >= len(dense):
            print("Reached end of data at index:", next_idx)
            return jsonify({"error": "End of data"}), 404

        row = dense.iloc[next_idx]
        candle = {
            'time': int(row.name.timestamp()),
            'open': float(row.open),
            'high': float(row.high),
            'low': float(row.low),
            'close': float(row.close)
        }

        # ✅ Add to preproc automatically
        preproc.add_candle(row)

        return jsonify({
            "next_idx": next_idx + 1,
            "candle": candle
        })


@app.route("/predict", methods=['POST'])
def predict():
    data = request.get_json(force=True)
    seq_len = data.get("seq_len")

    if not seq_len or not isinstance(seq_len, int):
        return jsonify({"error": "Provide 'seq_len' as an int"}), 400

    try:
        # prepare subsequence from current state
        seq_df = preproc.prepare_seq(seq_len)
    except ValueError as e:
        return jsonify({"error": str(e)}), 400

    X_np = seq_df[FEATURES].values.astype(np.float32)
    X_t = torch.from_numpy(X_np).unsqueeze(0)
    dict_x = {"main": X_t}
    with torch.no_grad():
        mdn_out = model(dict_x)

    pi    = mdn_out['pi'][0].cpu().numpy()
    mu    = mdn_out['mu'][0].cpu().numpy()
    sigma = mdn_out['sigma'][0].cpu().numpy()
    last_close = preproc.reference_dataset.iloc[-1]['close']
    return jsonify({
        'pred_prices': (last_close * mu).tolist(),
        'pred_sigmas': (last_close * sigma).tolist(),
        'pi': pi.tolist()
    })

if __name__ == '__main__':
    app.run(debug=True, use_reloader=False)
